In [12]:
import pandas as pd
import ast
from collections import Counter
from collections import defaultdict
import numpy as np

In [13]:
#import files
df = pd.read_csv('data/tmdb_5000_credits.csv')

In [14]:
#Cast cols to list/dict
df['cast'] = df['cast'].apply(ast.literal_eval)
df['crew'] = df['crew'].apply(ast.literal_eval)

In [15]:

#filter dictionaries
def cast_filter_fun(cast_list):
        updated_cast_list = []
        for cast_dictionary in cast_list:
                cast_filter = lambda x,y: dict([(i,x[i]) for i in x if i in set(y)   ])
                cast_filter_list = ['id', 'name','gender','order']
                ordered_cast_dict = {k: cast_filter(cast_dictionary, cast_filter_list)[k] for k in cast_filter_list}
                updated_cast_list.append(ordered_cast_dict)
        return updated_cast_list

In [16]:
#create gender ratio column
def get_gender_ratio(ls):
    male = 0
    female = 0
    noidea = 0
    for i in ls:
        if i['gender'] != 0:
            if i['gender'] == 1:
                female += 1
            else:
                male += 1
        else:
            noidea += 1
    new_dict = {'Gender 0':noidea, 'Gender 1': female, 'Gender 2': male}
    return new_dict

In [17]:
#create crew filter

def crew_filter_fun(cast_list):
        updated_cast_list = []
        for cast_dictionary in cast_list:
                cast_filter = lambda x,y: dict([(i,x[i]) for i in x if i in set(y)   ])
                cast_filter_list = ['id', 'name','department','gender']
                ordered_cast_dict = {k: cast_filter(cast_dictionary, cast_filter_list)[k] for k in cast_filter_list}
                updated_cast_list.append(ordered_cast_dict)
        return updated_cast_list

In [18]:
#department sizes
def get_department_counts(dict_list):
    if not dict_list:
        return {}
    
    department_ids = defaultdict(set)
    
    for d in dict_list:
        department = d.get('department')
        crew_id = d.get('id')
        
        if department is not None and crew_id is not None:
            department_ids[department].add(crew_id)
    
    department_counts = {dept: len(ids) for dept, ids in department_ids.items()}
    return department_counts

In [19]:
df['Adjusted_Cast'] = df['cast'].apply(cast_filter_fun)
df['Cast_Size'] = df['Adjusted_Cast'].apply(len)
df['Top_Cast'] = df['Adjusted_Cast'].apply(lambda x: x[:10])
df['Cast_Gender_Ratio'] = df['Adjusted_Cast'].apply(get_gender_ratio)

In [20]:
df['Adjusted_Crew'] = df['crew'].apply(crew_filter_fun)
df['Crew_Size'] = df['Adjusted_Crew'].apply(len)
df['Crew_Dept_Cout'] = df['Adjusted_Crew'].apply(get_department_counts)
df['Crew_Gender_Ratio'] = df['Adjusted_Crew'].apply(get_gender_ratio)

In [21]:
df['Cast_Crew_Ratio'] = df.apply(lambda row: row['Cast_Size'] / row['Crew_Size'] if row['Crew_Size'] else np.nan, axis=1)

In [22]:
df.to_csv('data/Adjusted_tmdb_credits.csv', index=False)